<a href="https://colab.research.google.com/github/enzocfb/general/blob/main/eda_2023_deductivos_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Analisis de Deductivos para analizar causas
Impacto :

input:
      Consolidato total de Evaluados por UGS(Formato 06) para todos los años
        Consolidado que contiene los portafolios : 2024-02 , 2024-01, 2023-01 , 2022-02

      Deductivos reportados por UGT
      Fechas referenciales por grupos

output:
      Consolidado de evaluados, y su estado : "Deductivo", y "No Deductivo"

"""

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
####################################################3
# Input
####################################################3

# Archivo completo de las evaluaciones realizadas por UGS, a partir de la cual se extrae la informacion del beneficiario,
# # Alcance : REGISTRADO- desde que se inicia en ESTADO REGISTRADO, LUEGO A ESTADO EVALUADO , luego  ESTADO VIABILIDAD y finalmente # a ESTADO DE DEDUCTIVIDAD
# KEY or INDICE del ARCHIVO:   CUV (Codigo que contiene el UIBGEO del CENTRO POBLADO + DNI del JEFE + AñO + NRO PORTAFOLIO)
#                              CONVENIO,  es una agrupacion de Beneficiarios (FALTA, no tiene en este momento esa data)

file = '/content/drive/MyDrive/@@@Dev/mvcs/Data/Formato06Completo.xlsx'

# 2. ARCHIVO DE DEDUCTIVOS (BENEFICIARIO QUE ESTA EN ESTADO = "DEDUCTIVO", es aquel que "SE EXCLUYE" DEL PROGRAMA. Este archivo es recibido de  UGT que contiene la informacion de todos aquellos que fueron
# ALCANCE : VIABLE-->DEDUCTIVO
# KEY or INDICE del ARCHIVO:
#                              CUV (Codigo que contiene el UIBGEO del CENTRO POBLADO + DNI del JEFE + AñO + NRO PORTAFOLIO) (SI TIENE)
#                              CONVENIO,  es una agrupacion de Beneficiarios,                                               (SI TIENE)

file_deductivos= '/content/drive/MyDrive/@@@Dev/mvcs/Data/LISTADO DE DEDUCTIVOS A (UGS).xlsx'


# Informacion a nivel de nucleos de las fechas de exp tecnico y fecha de Acta de inicio de Construccion
# Alcance : REGISTRADO-->EVALUADO-->VIABLE (TODOs: ESTADO_VIABLE = "VIABLE", INCLUIDO ESTADO_DEDUCTIVO = {"DEDUCTIVO", "NO DEDUCTIVO"] se une a nivel con archivo 2, con los
# 17 nov : Solo se tienen info para VIABLE-->DEDUCTIVO, falta a nivel de todos los VIABLES.
# KEY or INDICE del ARCHIVO:   CONVENIO,  es una agrupacion de Beneficiarios, (SI TIENE)

file_input_nucleo = '/content/drive/MyDrive/@@@Dev/mvcs/Data/NUCLEO - REPORTE COMPLEMENTARIO.xlsx'
file_input_2020_02 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/Pnvr_formato06_20231122-15.57.07.xlsx'

file_base_total_consolidado = '/content/drive/MyDrive/@@@Dev/mvcs/Data/Base Total Usuarios del PNVR_v7_20.11.23.xlsx'



####################################################3
# Output
####################################################3
# output temporales
file_output_1 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/CUVs no encontrados.xlsx'
file_output_2 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/CUVs si encontrados.xlsx'


file_temp_1 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/temp_1.xlsx'
file_temp_2 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/temp_2.xlsx'
file_output_3 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/temp_3.xlsx'

# output
file_output_analisis = '/content/drive/MyDrive/@@@Dev/mvcs/Data/deductivos-completo-analysis.xlsx'


# Constantes
constante_origen_PP068 = 'PP068'   # NUMERO DEL PROGRAMA ORIGEN

# Creamos ESTADOS : REGISTRADO, EVALUACION,  VIABILIDAD,
constante_estado_registrado = 'REGISTRADO'

constante_estado_evaluacion_evaluado = "EVALUADO"
constante_estado_evaluacion_rechaza = "RECHAZA_EVALUACION"
constante_estado_evaluacion_otros = 'OTROS_NO_EVALUADO'

constante_estado_viabilidad_viable = "VIABLE"
constante_estado_viabilidad_no_viable = "NO VIABLE"
constante_estado_sininfo = "SIN INFORMACION"
constante_estado_viabilidad_social_viable = 'SOCIALMENTE VIABLE'
constante_estado_viabilidad_social_no_viable = 'NO VIABLE SOCIALMENTE'

constante_estado_SI = "SI"
constante_estado_SI = "NO"


constante_estado_deductivo = "DEDUCTIVO"
constante_estado_no_deductivo = "NO_DEDUCTIVO"

# Mejoras
# Definir Estructura UGS
# Definir Estructura UGS + UGT




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def convert_to_date(value):
    try:
        return pd.to_datetime(value)
    except ValueError:
        # Handle the case where parsing the date fails
        return value

# # Read the Excel file with the converters parameter
# df = pd.read_excel(file, converters={"Your_Column_Name": convert_to_date})

In [ ]:
# La hoja excel, original se le transforma para poder ser leida
# Combinar celdas
# subir fila 6,
# bajamos los nombre de capos de la fila 5 a fila 6 (solo los que faltan)


df = pd.read_excel(file, converters={"CODIGO_CCPP": str, "UBIGEO": str})
# df = pd.read_excel(file, converters={"CODIGO_CCPP": str, "UBIGEO": str, "FEC_EVALUACION": convert_to_date})
df.head(3)

# CREACION DE ESTADOS PARA TABLA INICIAL



In [ ]:
df.info()

In [ ]:
# Seleccionar campos requeridos para el analisis y construir las features

columns_selected_df = ['NUMERO', 'ORIGEN', 'PORTAFOLIO_T', 'CUV', 'CODIGO_CCPP', 'UBIGEO', 'DEPARTAMENTO','PROVINCIA', 'DISTRITO', 'CENTRO_POBLADO', 'COMUNIDAD', 'PATERNO_JEFE_HOGAR', 'NOMBRES_JEFE_HOGAR', 'DNI_JEFE_HOGAR', 'SEXO_JEFE_HOGAR', 'EDAD_JEFE_HOGAR', 'FEC_NAC_JEFE_HOGAR', 'CANT', 'CTD_DISCAPACIDAD', 'ALTURA', 'LATITUD', 'LONGITUD', 'CLASIFICACION_SOCIOECONOMICA_CSE', 'VIABILIDAD_TOTAL', 'VIABILIDAD_SOCIAL', 'VIABILIDAD_TECNICA','DNI_CYG','ACEPTA_ENTREVISTA', 'CRITERIO_SOCIAL', 'CRITERIO_TECNICO', 'FEC_EVALUACION', 'FECHA_CONSULTA_CSE']
df = df[columns_selected_df]

# renonmbar dni coyg por DNI_CYG

columns_selected_df_2020_02 = ['CODIGO CUV', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO',
															'CENTRO POBLADO', 'COMUNIDAD', 'PATERNO', 'MATERNO', 'NOMBRES', 'DNI', 'SEXO',
                               'ALTURA','LATITUD', 'LONGITUD', 'DETERMINACIÓN DE LA VIABILIDAD', 'CRITERIO SOCIAL', 'CRITERIO TÉCNICO',
															'VIABILIDAD SOCIAL', 'VIABILIDAD TECNICA',
															'DNI_CYG',
															'CLASIFICACION SOCIOECONOMICA CSE'
															]



In [ ]:
############################################################3
# Crear base unica, a partir de base consolidada y formato06
###############################################################################


In [ ]:
# df_deductivos_faltante = rows_not_in_df06

df = pd.read_excel(file)
df = df[columns_selected_df]

# el campo CUV, tiene vacios, pero el campo DNI, no asi que lo elegimos como clave
# para ello verifricamos si este campo tiene duplicados




In [ ]:
# Counting duplicated values in the 'Name' column
duplicates_count = df.duplicated(subset='DNI_JEFE_HOGAR').sum()

# Displaying the DataFrame and the count of duplicated values
# print("Original DataFrame:")
# print(df.head(2))

print("\nCount of duplicated values in the 'Name' column:", duplicates_count)
# Count of duplicated values in the 'Name' column: 6737

# Se descarta dni como clave y solo se usa CUV, para ello elinamos los CUV que estan vacios



In [ ]:
print(df.shape[0])
# Dropping rows where the 'Name' column is null
df_cleaned = df.dropna(subset=['CUV'], inplace = True)
print(df.shape[0])

In [ ]:
df_base_consolidada = pd.read_excel(file_base_total_consolidado)
df_base_consolidada.head(3)

columns_selected_df_base = [ 'OBJECTID', 'CUV', 'Ubigeo CP', 'Ubigeo', 'Departamento', 'Provincia', 'Distrito', 'Centro Poblado', 'Comunidad', 'Ape. Paterno', 'Ape. Materno', 'Nombre', 'DNI', 'Sexo', 'edad', 'Fecha de Nacimiento',
                            'Número de Miembros', 'altitud', 'latitud', 'longitud']

new_columns_to_add = ['NUMERO', 'ORIGEN', 'PORTAFOLIO_T', 'CLASIFICACION_SOCIOECONOMICA_CSE', 'VIABILIDAD_TOTAL', 'VIABILIDAD_SOCIAL', 'VIABILIDAD_TECNICA',
                      'ACEPTA_ENTREVISTA', 'CRITERIO_SOCIAL', 'CRITERIO_TECNICO', 'FEC_EVALUACION', 'FECHA_CONSULTA_CSE']

df_base = df_base_consolidada[columns_selected_df_base]

In [ ]:
  # Add empty columns to the DataFrame
  for col_name in new_columns_to_add:
      df_base[col_name] = None

In [ ]:
df_base.info()

In [ ]:
# ================================================================
# renombar las columnas campos segun el df padre : df:
# ================================================================

df_base.rename(columns={'Ubigeo CP': 'CODIGO_CCPP'}, inplace=True)
df_base.rename(columns={'Ubigeo': 'UBIGEO'}, inplace=True)
df_base.rename(columns={'Departamento': 'DEPARTAMENTO'}, inplace=True)
df_base.rename(columns={'Provincia': 'PROVINCIA'}, inplace=True)
df_base.rename(columns={'Distrito': 'DISTRITO'}, inplace=True)
df_base.rename(columns={'Centro Poblado': 'CENTRO_POBLADO'}, inplace=True)
df_base.rename(columns={'Comunidad': 'COMUNIDAD'}, inplace=True)
df_base.rename(columns={'Ape. Paterno': 'PATERNO_JEFE_HOGAR'}, inplace=True)
df_base.rename(columns={'Ape. Materno': 'MATERNO_JEFE_HOGAR'}, inplace=True)
df_base.rename(columns={'Nombre': 'NOMBRES_JEFE_HOGAR'}, inplace=True)

df_base.rename(columns={'DNI': 'DNI_JEFE_HOGAR'}, inplace=True)
df_base.rename(columns={'Sexo': 'SEXO_JEFE_HOGAR'}, inplace=True)
df_base.rename(columns={'edad': 'EDAD_JEFE_HOGAR'}, inplace=True)
df_base.rename(columns={'Fecha de Nacimiento': 'FEC_NAC_JEFE_HOGAR'}, inplace=True)
df_base.rename(columns={'Número de Miembros': 'CANT'}, inplace=True)
df_base.rename(columns={'altitud': 'ALTURA'}, inplace=True)
df_base.rename(columns={'latitud': 'LATITUD'}, inplace=True)
df_base.rename(columns={'longitud': 'LONGITUD'}, inplace=True)

In [ ]:
df_base_with_f06_inner = pd.merge(df_base, df, on = 'CUV', suffixes={'_BASE', '_F06'})

In [ ]:
df_base_with_f06_inner.to_excel(file_temp_1)

In [ ]:
df_base_with_f06_outer = pd.merge(df_base, df, on = 'CUV', how = 'outer', suffixes={'_BASE', '_F06'})

In [ ]:
df_base_with_f06_outer.to_excel(file_temp_2)

In [ ]:
df_base_with_f06_inner.head()

In [ ]:
print(df_base_with_f06_inner.shape[0])
print(df_base_with_f06_outer.shape[0])

In [ ]:
  # =============================================================================================
  # POST : Crear los Estados
  #================================================================================================

  # reglas para llenar los campos faltantes, derivados de informacion existente
  # Se llenan de esta forma, porque ya son DEDUCTIVOS, es decir han pasado por todas las etapas desde ser registrados, evaluados, y viables.
  # viabilidad, considerando que todos los deductivos, previaente fueron REGISTRADOS, luego EVALUADOS, y luego viables, entonces se coloca el valor VIABLE , lara todos los registros
  #

  df_deductivos_en_base['ESTADO_REGISTRADO'] = constante_estado_registrado
  df_deductivos_en_base['ESTADO_EVALUACION'] = constante_estado_evaluacion_evaluado
  df_deductivos_en_base['ESTADO_VIABLE'] = constante_estado_viabilidad_viable
  df_deductivos_en_base['ESTADO_DEDUCTIVO'] = constante_estado_deductivo


  df_deductivos_en_base['VIABILIDAD_TOTAL'] = constante_estado_viabilidad_viable
  df_deductivos_en_base['VIABILIDAD_SOCIAL'] = constante_estado_viabilidad_social_viable
  df_deductivos_en_base['VIABILIDAD_TECNICA'] = constante_estado_viabilidad_viable


  # Campos adicionados con logica



  # ============================================





In [ ]:
# ver que otros campos de deductivos, deben ser tomados para completar segun salida de analisis?_deductivos.xlsx


In [ ]:
import pandas as pd

# Creating two sample DataFrames
df1 = pd.DataFrame({'key': ['A', 'B', 'C', 'D'],
                    'value': [1, 2, 3, 4]})

df2 = pd.DataFrame({'key': ['B', 'D', 'E', 'F'],
                    'value': [5, 6, 7, 8]})


print(df1)

print(df2)


In [ ]:
# Inner merge
inner_merged = pd.merge(df1, df2, how='inner', on='key')
print("Inner Merge:")
print(inner_merged)

# Left merge
left_merged = pd.merge(df1, df2, how='left', on='key')
print("\nLeft Merge:")
print(left_merged)

# Right merge
right_merged = pd.merge(df1, df2, how='right', on='key')
print("\nRight Merge:")
print(right_merged)

# Outer merge
outer_merged = pd.merge(df1, df2, how='outer', on='key')
print("\nOuter Merge:")
print(outer_merged)

In [ ]:
import pandas as pd

# Creating a sample DataFrame
data = {'Name': ['Alice', 'Alice', 'Charlie', 'Alice', 'David'],
        'Age': [25, 30, 35, 25, 40],
        'City': ['New York', 'San Francisco', 'Los Angeles', 'New York', 'Seattle']}

df3 = pd.DataFrame(data)

df3


In [ ]:
# Counting duplicated values in the 'Name' column
duplicates_count = df3.duplicated(subset='Name').sum()

# Displaying the DataFrame and the count of duplicated values
print("Original DataFrame:")
print(df3)

print("\nCount of duplicated values in the 'Name' column:", duplicates_count)